In [ ]:
from pandas import read_csv, DataFrame, Series, Period, Index
from matplotlib.pyplot import figure, show
from dslabs_functions import plot_line_chart, HEIGHT

# File details
file_tag = "forecast_gdp_europe"
target = "GDP"

# Load data from CSV
data: DataFrame = read_csv(
    "/Users/tomifemme/Desktop/DataScience/forecast_gdp_europe.csv",
    index_col="Year",  # Adjust to 'Year' if that's the correct column name
    sep=",",  # Comma-separated values
    decimal=".",  # Use period as decimal separator
    parse_dates=["Year"],  # Parse the 'Year' column as datetime
    infer_datetime_format=True,  # Automatically infer datetime format
)

# Extract target series
series: Series = data[target]

# Profiling output
print("Number of Records =", series.shape[0])
print("First Year:", series.index[0].year)
print("Last Year:", series.index[-1].year)

def ts_aggregation_by(
    data: Series | DataFrame,
    gran_level: str = "Y",  # Granularity level ('Y', '5Y', '10Y')
    agg_func: str = "mean",  # Aggregation function (e.g., mean)
) -> Series | DataFrame:
    df: Series | DataFrame = data.copy()

    # Convert the index to a Period index
    if gran_level == 'Y':
        index: Index[Period] = df.index.to_period('Y') 
    elif gran_level == '5Y':
        index: Index[Period] = df.index.to_period('5Y') 
    elif gran_level == '10Y':
        index: Index[Period] = df.index.to_period('10Y') 
    else:
        raise ValueError(f"Unsupported granularity level: {gran_level}")

    # Apply the shift based on granularity
    if gran_level == '5Y':
        start_year = df.index[0].year 
        index = index.shift(-(start_year % 5)) 
    elif gran_level == '10Y':
        start_year = df.index[0].year
        index = index.shift(-(start_year % 10))

    # Group by the Period index and apply the aggregation function
    df = df.groupby(by=index, dropna=True, sort=True).agg(agg_func)

    # Convert the Period index back to a timestamp for plotting
    df.index = df.index.to_timestamp()

    return df

# Define granularity levels to study
grans: list[str] = ["Y", "5Y", "10Y"]  # Yearly, 5-Year, 10-Year

# Loop through each granularity level and plot one by one
for granularity in grans:
    # Aggregate the time series to the current granularity level
    ss: Series = ts_aggregation_by(series, granularity)
    
    # Print the aggregated data for debugging
    print(f"\nAggregated data for granularity {granularity}:")
    print(ss)

    # Create a figure for the current granularity
    figure(figsize=(3 * HEIGHT, HEIGHT / 2))
    
    # Plot the aggregated time series
    plot_line_chart(
        ss.index.to_list(),
        ss.to_list(),
        xlabel=f"Year ({granularity})",  # Label with granularity level
        ylabel=target,
        title=f"granularity={granularity}",  # Title for the plot
    )

    # Display the plot one by one
    show()

SyntaxError: invalid syntax (3806945650.py, line 27)